##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 単語埋め込み (Word embeddings)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/word_embeddings">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ja/tutorials/text/word_embeddings.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ja/tutorials/text/word_embeddings.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/ja/tutorials/text/word_embeddings.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Note: これらのドキュメントは私たちTensorFlowコミュニティが翻訳したものです。コミュニティによる 翻訳は**ベストエフォート**であるため、この翻訳が正確であることや[英語の公式ドキュメント](https://www.tensorflow.org/?hl=en)の 最新の状態を反映したものであることを保証することはできません。 この翻訳の品質を向上させるためのご意見をお持ちの方は、GitHubリポジトリ[tensorflow/docs](https://github.com/tensorflow/docs)にプルリクエストをお送りください。 コミュニティによる翻訳やレビューに参加していただける方は、 [docs-ja@tensorflow.org メーリングリスト](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ja)にご連絡ください。

このチュートリアルでは、単語埋め込みを紹介します。このチュートリアルには、小さいデータセットを使って単語埋め込みを最初から学習させ、その埋め込みベクトルを [Embedding Projector](http://projector.tensorflow.org) （下図参照）を使って可視化するためのプログラムがすべて含まれています。

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding.jpg?raw=1" alt="Screenshot of the embedding projector" width="400"/>

## テキストを数値で表す

機械学習モデルは、ベクトル（数値の配列）を入力として受け取ります。テキストを扱う際、最初に決めなければならないのは、文字列を機械学習モデルに入力する前に、数値に変換する（あるいはテキストを「ベクトル化」する）ための戦略です。このセクションでは、これを行う3つの戦略を見てみます。

### ワンホット・エンコーディング

最初のアイデアとして、ボキャブラリーの中の単語それぞれを「ワンホット」エンコードするというのがあります。 "The cat sat on the mat" という文を考えてみましょう。この文に含まれるボキャブラリー（ユニークな単語）は、 (cat, mat, on, sat, the) です。それぞれの単語を表現するため、ボキャブラリーの長さに等しいゼロベクトルを作り、その単語に対応するインデックスの場所に 1 を立てます。これを下図で示します。 

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/one-hot.png?raw=1" alt="Diagram of one-hot encodings" width="400" />

文をエンコードしたベクトルを作成するには、その後、それぞれの単語のワンホット・ベクトルをつなげればよいのです。

Key point: この手法は非効率です。ワンホット・エンコードされたベクトルは疎（つまり、ほとんどのインデックスではゼロ）です。ボキャブラリーに 10,000 の単語があると考えてみましょう。単語をすべてワンホット・エンコードするということは、要素の 99.99% がゼロであるベクトルを作ることになります。

### それぞれの単語をユニークな数値としてエンコードする

2つ目のアプローチとして、それぞれの単語をユニークな数値でエンコードするというのがあります。上記の例をとれば、"cat" に 1、"mat" に 2、というふうに番号を割り当てることができます。そうすれば、 "The cat sat on the mat" という文は、 [5, 1, 4, 3, 5, 2] という密なベクトルで表すことができます。この手法は効率的です。疎なベクトルの代わりに、密な（すべての要素が入っている）ベクトルが得られます。

しかしながら、このアプローチには 2つの欠点があります。

* 整数エンコーディングは勝手に決めたものです（単語間のいかなる関係性も含んでいません）。

* 整数エンコーディングはモデルにとっては解釈しにくいものです。たとえば、線形分類器はそれぞれの特徴量について単一の重みしか学習しません。したがって、2つの単語が似かよっていることと、それらのエンコーディングが似かよっていることの間には、なんの関係もありません。この特徴と重みの組み合わせには意味がありません。

### 単語埋め込み

単語埋め込みを使うと、似たような単語が似たようにエンコードされる、効率的で密な表現が得られます。重要なのは、このエンコーディングを手動で行う必要がないということです。埋め込みは浮動小数点数の密なベクトルです（そのベクトルの長さはあなたが指定するパラメータです）。埋め込みベクトルの値は指定するものではなく、学習されるパラメータです（モデルが密結合レイヤーの重みを学習するように、訓練をとおしてモデルが学習する重みです）。一般的には、（小さいデータセットの場合の）8次元の埋め込みベクトルから、大きなデータセットを扱う 1024次元のものまで見られます。高次元の埋め込みは単語間の細かな関係を取得できますが、学習にはよりたくさんのデータが必要です。

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding2.png?raw=1" alt="Diagram of an embedding" width="400"/>

上図は単語埋め込みを図示したものです。それぞれの単語が 4次元の浮動小数点数のベクトルで表されています。埋め込みは「参照テーブル」と考えることもできます。重みが学習された後では、テーブルを参照して、それぞれの単語を対応する密ベクトルにエンコードできます。

## 設定

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version は Colab 中でのみ使用できます
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

## Embedding レイヤーを使う

Keras では単語埋め込みを使うのも簡単です。[Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) レイヤーを見てみましょう。

Embedding レイヤーは、（特定の単語を示す）整数のインデックスに（その埋め込みである）密なベクトルを対応させる参照テーブルとして理解することができます。埋め込みの次元数（あるいはその幅）は、取り組んでいる問題に適した値を実験して求めるパラメータです。これは、Dense レイヤーの中のニューロンの数を実験で求めるのとまったくおなじです。

In [4]:
embedding_layer = layers.Embedding(1000, 32)

Embedding レイヤーを作成するとき、埋め込みの重みは（ほかのレイヤーとおなじように）ランダムに初期化されます。訓練を通じて、これらの重みはバックプロパゲーションによって徐々に調整されます。いったん訓練が行われると、学習された単語埋め込みは、（モデルを訓練した特定の問題のために学習された結果）単語の間の類似性をおおまかにコード化しています。

Embedding レイヤーに整数を渡すと、結果はそれぞれの整数が埋め込みテーブルのベクトルに置き換えられます。

In [5]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[ 0.01536964, -0.02893396,  0.00805476, -0.00879277, -0.02329649,
         0.01090913,  0.00075917,  0.03399904, -0.04283867, -0.03334184,
        -0.03518311,  0.01216765, -0.04280223, -0.00485247,  0.01175017,
        -0.01758611,  0.01495043,  0.0357315 ,  0.04665111,  0.0044551 ,
        -0.03676478,  0.00683149,  0.02648311, -0.02317376, -0.02922149,
         0.01651103,  0.01730004,  0.02771946,  0.00135338,  0.03408105,
        -0.01189137, -0.00981994],
       [ 0.04893507, -0.02704159, -0.01636837, -0.04495286, -0.00277602,
         0.00679402, -0.01419598,  0.02915657, -0.04149633, -0.02538633,
        -0.0041279 , -0.04128667,  0.03065732, -0.00550362,  0.0030761 ,
         0.00652701,  0.02318848,  0.04257066, -0.00777096,  0.0117504 ,
        -0.00113489,  0.03469534,  0.03195261,  0.0362216 , -0.0091609 ,
        -0.04703332,  0.00637571, -0.03441916, -0.00659513, -0.00443795,
         0.00155725, -0.00197284],
       [-0.01873819, -0.03550842,  0.03499896, -0.0286

テキストあるいはシーケンスの問題では、入力として、Embedding レイヤーは shape が `(samples, sequence_length)` の2次元整数テンソルを取ります。ここで、各エントリは整数のシーケンスです。このレイヤーは、可変長のシーケンスを埋め込みベクトルにすることができます。上記のバッチでは、 `(32, 10)` （長さ10のシーケンス32個のバッチ）や、 `(64, 15)` （長さ15のシーケンス64個のバッチ）を埋め込みレイヤーに投入可能です。

シーケンスのバッチを入力されると、Embedding レイヤーは shape が `(samples, sequence_length, embedding_dimensionality)` の3次元浮動小数点数テンソルを返します。この可変長のシーケンスを、固定長の表現に変換するには、さまざまな標準的なアプローチが存在します。Dense レイヤーに渡す前に、RNNやアテンション、プーリングレイヤーを使うことができます。ここでは、一番単純なのでプーリングを使用します。[RNN
 を使ったテキスト分類](https://github.com/tensorflow/docs/blob/master/site/ja/tutorials/text/text_classification_rnn.ipynb) は次のステップとして有効です。

## 埋め込みを最初から学習する

IMDB の映画レビューの感情分析器を訓練しようと思います。そのプロセスを通じて、埋め込みを最初から学習します。ここでは、前処理済みのデータセットを使用します。

テキストデータセットを最初からロードする方法については、[テキスト読み込みのチュートリアル](../load_data/text.ipynb)を参照してください。

In [6]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

エンコーダー（`tfds.features.text.SubwordTextEncoder`）を取得し、すこしボキャブラリを見てみましょう。

ボキャブラリ中の "\_" は空白を表しています。ボキャブラリの中にどんなふうに（"\_"）で終わる単語全体と、より長い単語を構成するための単語の部分が含まれているかに注目してください。

In [7]:
encoder = info.features['text'].encoder
encoder.subwords[:20]

['the_',
 ', ',
 '. ',
 'a_',
 'and_',
 'of_',
 'to_',
 's_',
 'is_',
 'br',
 'in_',
 'I_',
 'that_',
 'this_',
 'it_',
 ' /><',
 ' />',
 'was_',
 'The_',
 'as_']

映画のレビューはそれぞれ長さが異なっているはずです。`padded_batch` メソッドを使ってレビューの長さを標準化します。

In [8]:
padded_shapes = ([None],())
train_batches = train_data.shuffle(1000).padded_batch(10, padded_shapes = padded_shapes)
test_batches = test_data.shuffle(1000).padded_batch(10, padded_shapes = padded_shapes)

インポートした状態では、レビューのテキストは整数エンコードされています（それぞれの整数がボキャブラリ中の特定の単語あるいは部分単語を表しています）。

あとの方のゼロに注目してください。これは、バッチが一番長いサンプルに合わせてパディングされた結果です。

In [9]:
train_batch, train_labels = next(iter(train_batches))
train_batch.numpy()

array([[  19, 1372, 1150, ...,    0,    0,    0],
       [ 134,   12,   81, ...,    0,    0,    0],
       [  20,  278,   14, ...,   53, 2419, 7975],
       ...,
       [ 131,   56,  245, ...,    0,    0,    0],
       [  12,  258,  335, ...,    0,    0,    0],
       [  12,  723,  164, ...,    0,    0,    0]])

### 単純なモデルの構築

[Keras Sequential API](../../guide/keras) を使ってモデルを定義することにします。今回の場合、モデルは「連続した Bag of Words」スタイルのモデルです。

* 次のレイヤーは Embedding レイヤーです。このレイヤーは整数エンコードされた語彙を受け取り、それぞれの単語のインデックスに対応する埋め込みベクトルをみつけて取り出します。これらのベクトルはモデルの訓練により学習されます。このベクトルは出力配列に次元を追加します。その結果次元は `(batch, sequence, embedding)` となります。

* 次に、GlobalAveragePooling1D レイヤーが、それぞれのサンプルについて、シーケンスの次元で平均を取り、固定長の出力ベクトルを返します。これにより、モデルは可変長の入力を最も簡単な方法で扱えるようになります。

* 最後のレイヤーは、1個の出力ノードを持つ Dense レイヤーです。シグモイド活性化関数を使うことで、値は 0 と 1 の間の値を取り、レビューがポジティブ（好意的）であるかどうかの確率（または確信度）を表します。

Caution: このモデルはマスキングを使用していません。このため、ゼロパディングが入力の一部として扱われ、結果としてパディングの長さが出力に影響を与える可能性があります。これを修正するには[マスキングとパディングのガイド](../../guide/keras/masking_and_padding)を参照してください。

In [10]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 130,977
Trainable params: 130,977
Non-trainable params: 0
_________________________________________________________________


### モデルのコンパイルと訓練

In [11]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
  train_batches,
  epochs=10,
  validation_data=test_batches, validation_steps=20)

Epoch 1/10
2500/2500 [==============================] - 25s 10ms/step - loss: 0.6362 - accuracy: 0.6877 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
2500/2500 [==============================] - 15s 6ms/step - loss: 0.4662 - accuracy: 0.8361 - val_loss: 0.4055 - val_accuracy: 0.8400
Epoch 3/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.3629 - accuracy: 0.8788 - val_loss: 0.3477 - val_accuracy: 0.8800
Epoch 4/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.3094 - accuracy: 0.8954 - val_loss: 0.3142 - val_accuracy: 0.8500
Epoch 5/10
2500/2500 [==============================] - 15s 6ms/step - loss: 0.2737 - accuracy: 0.9071 - val_loss: 0.2537 - val_accuracy: 0.9250
Epoch 6/10
2500/2500 [==============================] - 15s 6ms/step - loss: 0.2500 - accuracy: 0.9152 - val_loss: 0.2567 - val_accuracy: 0.9100
Epoch 7/10
2500/2500 [==============================] - 15s 6ms/step - loss: 0.2324 - accuracy: 0.9211 - val_loss: 0.3082

このアプローチにより、モデルの評価時の正解率は 88% 前後に達します（モデルは過学習しており、訓練時の正解率の方が際立って高いことに注意してください）。

In [12]:
import matplotlib.pyplot as plt

history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,9))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim((0.5,1))
plt.show()

<Figure size 1200x900 with 1 Axes>

<Figure size 1200x900 with 1 Axes>

## 学習した埋め込みの取得

次に、訓練によって学習された単語埋め込みを取得してみます。これは、shape が `(vocab_size,embedding-dimension)` の行列になります。

In [13]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(8185, 16)


この重みをディスクに出力します。[Embedding Projector](http://projector.tensorflow.org) を使うため、タブ区切り形式の2つのファイルをアップロードします。（埋め込みを含む）ベクトルのファイルと、（単語を含む）メタデータファイルです。

In [14]:
import io

encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] # 0 はパディングのためスキップ
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

このチュートリアルを [Colaboratory](https://colab.research.google.com) で実行している場合には、下記のコードを使ってこれらのファイルをローカルマシンにダウンロードすることができます（あるいは、ファイルブラウザーを使います。*View -> Table of contents -> File browser* ）。

In [15]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

## 埋め込みを可視化する

埋め込みを可視化するため、これらのファイルを Embedding Projector にアップロードします。

[Embedding Projector](http://projector.tensorflow.org/) を開きます。

* "Load data" をクリックします

* 上記で作成した```vecs.tsv``` と ```meta.tsv``` の 2つのファイルをアップロードします


学習させた埋め込みが表示されます。単語を探し、最も近い単語を見つけることができます。たとえば、"beautiful" という単語を探してみてください。近くに、 "wonderful" のような単語が見つかると思います。

Note: 訓練の前に重みが乱数によってどのように初期化されたかによって、結果は少し異なるかもしれません。

Note: 実験として、もっと単純なモデルを使い、より解釈しやすい埋め込みを作ることもできます。`Dense(16)` レイヤーを削除してみてください。このモデルを再度訓練して、埋め込みの可視化をもう一度行ってみましょう。

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding.jpg?raw=1" alt="Screenshot of the embedding projector" width="400"/>


## 次のステップ

このチュートリアルでは、小さなデータセットを使い、単語埋め込みを最初から訓練し、可視化する方法を見てきました。

* リカレントネットワークについて学ぶには、[Keras RNN ガイド](../../guide/keras/rnn.ipynb) を参照してください。

* テキスト分類について更に学ぶには、（全体のワークフローや、どういうときに埋め込みあるいはワンホットエンコーディングを使うべきかについて興味があれば）この実践的なテキスト分類の [ガイド](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) を推奨します。